In [1]:
from typing import List
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.schema import BaseNode, TransformComponent
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.core.text_splitter import SentenceSplitter
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core import Settings
from langchain_community.llms import Ollama
import faiss
import os
import sys
from dotenv import load_dotenv

In [2]:
EMBED_DIMENSION = 512

# llamaindex ==> length of the tokens
CHUNK_SIZE = 200
CHUNK_OVERLAP = 50


In [3]:
Settings.embed_model = OllamaEmbedding( model_name="llama3.1:latest",)

In [4]:
Settings.embed_model

OllamaEmbedding(model_name='llama3.1:latest', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x1068b2aa0>, num_workers=None, base_url='http://localhost:11434', ollama_additional_kwargs={})

In [6]:
path = "data/"
node_parser = SimpleDirectoryReader(input_dir=path, required_exts=['.pdf'])
documents = node_parser.load_data()

In [7]:
# Create FaisVectorStore to store embeddings
faiss_index = faiss.IndexFlatL2(EMBED_DIMENSION)
vector_store = FaissVectorStore(faiss_index=faiss_index)

In [8]:
class TextCleaner(TransformComponent):
    """
    Transformation to be used within the ingestion pipeline.
    Cleans clutters from texts.
    """
    def __call__(self, nodes, **kwargs) -> List[BaseNode]:
        
        for node in nodes:
            node.text = node.text.replace('\t', ' ') # Replace tabs with spaces
            node.text = node.text.replace(' \n', ' ') # Replace paragraph seperator with spacaes
            
        return nodes

In [9]:
text_splitter = SentenceSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)

# Create a pipeline with defined document transformations and vectorstore
pipeline = IngestionPipeline(
    transformations=[
        TextCleaner(),
        text_splitter,
    ],
    vector_store=vector_store, 
)

In [10]:
pipeline

IngestionPipeline(name='default', project_name='Default', transformations=[TextCleaner(), SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x2a6632830>, id_func=<function default_id_func at 0x127a2b9a0>, chunk_size=200, chunk_overlap=50, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?')], documents=None, readers=None, vector_store=FaissVectorStore(stores_text=False, is_embedding_query=True), cache=IngestionCache(nodes_key='nodes', collection='llama_cache', cache=<llama_index.core.storage.kvstore.simple_kvstore.SimpleKVStore object at 0x2a768beb0>), docstore=None, docstore_strategy=<DocstoreStrategy.UPSERTS: 'upserts'>, disable_cache=False)

In [11]:
# Run pipeline and get generated nodes from the process
nodes = pipeline.run(documents=documents)

In [12]:
vector_store_index = VectorStoreIndex(nodes)
retriever = vector_store_index.as_retriever(similarity_top_k=2)

In [13]:
retriever

In [14]:
def show_context(context):
    """
    Display the contents of the provided context list.

    Args:
        context (list): A list of context items to be displayed.

    Prints each context item in the list with a heading indicating its position.
    """
    for i, c in enumerate(context):
        print(f"Context {i+1}:")
        print(c.text)
        print("\n")

In [15]:
test_query = "What is the main cause of climate change?"
context = retriever.retrieve(test_query)
show_context(context)

Context 1:
Most of these climate changes are attributed to very small variations in Earth's orbit that change the amount of solar energy our planet receives. During the Holocene epoch, which began at the end of the last ice age, human societies f lourished, but the industrial era has seen unprecedented changes.  Modern Observations  Modern scientific observations indicate a rapid increase in global temperatures, sea levels, and extreme weather events. The Intergovernmental Panel on Climate Change (IPCC) has documented these changes extensively. Ice core samples, tree rings, and ocean sediments provide a historical record that scientists use to understand past climate conditions and predict future trends. The evidence overwhelmingly shows that recent changes are primarily driven by human activities, particularly the emission of greenhou se gases.


Context 2:
The evidence overwhelmingly shows that recent changes are primarily driven by human activities, particularly the emission of gree

In [17]:
import json
from deepeval import evaluate
from deepeval.metrics import GEval, FaithfulnessMetric, ContextualRelevancyMetric
from deepeval.test_case import LLMTestCaseParams
from evaluation.evaluate_rag import create_deep_eval_test_cases

# Set llm model for evaluation of the question and answers 
LLM_MODEL = Ollama(model= "llama3.1:latest")

# Define evaluation metrics
correctness_metric = GEval(
    name="Correctness",
    model=LLM_MODEL,
    evaluation_params=[
        LLMTestCaseParams.EXPECTED_OUTPUT,
        LLMTestCaseParams.ACTUAL_OUTPUT
    ],
    evaluation_steps=[
        "Determine whether the actual output is factually correct based on the expected output."
    ],
)

faithfulness_metric = FaithfulnessMetric(
    threshold=0.7,
    model=LLM_MODEL,
    include_reason=False
)

relevance_metric = ContextualRelevancyMetric(
    threshold=1,
    model=LLM_MODEL,
    include_reason=True
)

def evaluate_rag(query_engine, num_questions: int = 5) -> None:
    """
    Evaluate the RAG system using predefined metrics.

    Args:
        query_engine: Query engine to ask questions and get answers along with retrieved context.
        num_questions (int): Number of questions to evaluate (default: 5).
    """
    
    
    # Load questions and answers from JSON file
    q_a_file_name = "data/q_a.json"
    with open(q_a_file_name, "r", encoding="utf-8") as json_file:
        q_a = json.load(json_file)

    questions = [qa["question"] for qa in q_a][:num_questions]
    ground_truth_answers = [qa["answer"] for qa in q_a][:num_questions]
    generated_answers = []
    retrieved_documents = []

    # Generate answers and retrieve documents for each question
    for question in questions:
        response = query_engine.query(question)
        context = [doc.text for doc in response.source_nodes]
        retrieved_documents.append(context)
        generated_answers.append(response.response)

    # Create test cases and evaluate
    test_cases = create_deep_eval_test_cases(questions, ground_truth_answers, generated_answers, retrieved_documents)
    evaluate(
        test_cases=test_cases,
        metrics=[correctness_metric, faithfulness_metric, relevance_metric]
    )

ValidationError: 1 validation error for ChatOpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)